In [2]:
class StridePrefetcher:
    """简单的步长预取器，基于 mvin 指令"""
    def __init__(self, stride=4):
        self.stride = stride  # 默认步长
        self.last_rs1_address = 0  # 记录上一次 mvin 指令的 rs1 地址

    def execute_mvin(self, rs1):
        """
        模拟 mvin 指令的执行，并根据步长进行预取。
        rs1: 虚拟 DRAM 地址
        rs2: scratchpad 或 accumulator 地址
        """
        # if self.last_rs1_address is not None:
        # 更新stride
        stride = rs1 - self.last_rs1_address
        self.set_stride(stride) 


        # 根据上一次的 rs1 地址预测下一次访问的 DRAM 地址
        predicted_address = rs1 + self.stride
        # print(f"{predicted_address} {self.last_rs1_address} {self.stride}")

        # 更新上一次访问的 rs1 地址
        self.last_rs1_address = rs1

        return predicted_address, stride

    def set_stride(self, new_stride):
        """动态调整步长"""
        self.stride = new_stride


if __name__ == "__main__":
    # 初始化步长预取器
    prefetcher = StridePrefetcher(stride=4)

    # 模拟多个 mvin 指令的执行，rs1 为虚拟 DRAM 地址，rs2 为 scratchpad 地址
    mvin_instructions = [
        1000,  # mvin rs1=1000, rs2=2000
        1004,  # mvin rs1=1004, rs2=2004
        1008,  # mvin rs1=1008, rs2=2008
        1012   # mvin rs1=1012, rs2=2012
    ]

    # 执行每条 mvin 指令
    for rs1 in mvin_instructions:
        predicted_address, stride = prefetcher.execute_mvin(rs1)
        print(f"本次访问的DRAM地址{rs1}, 设置步长{stride}, 预测下一次{predicted_address}")



本次访问的DRAM地址1000, 设置步长1000, 预测下一次2000
本次访问的DRAM地址1004, 设置步长4, 预测下一次1008
本次访问的DRAM地址1008, 设置步长4, 预测下一次1012
本次访问的DRAM地址1012, 设置步长4, 预测下一次1016


In [6]:
class StridePrefetcherGroup:
    """预取器组，包含多个步长预取器"""
    def __init__(self, num_prefetchers=16, default_stride=4):
        # 创建指定数量的步长预取器，每个预取器初始步长为默认值
        self.prefetchers = [StridePrefetcher(stride=default_stride) for _ in range(num_prefetchers)]

    def prefetch(self, mvin_instructions):
        """
        执行一组 mvin 指令，每个预取器处理一个。
        mvin_instructions: 每个元素是 rs1 ，表示 mvin 指令
        """
        # 确保 mvin 指令的数量和预取器数量一致
        assert len(mvin_instructions) == len(self.prefetchers), "mvin 指令数量应与预取器数量相等"

        # 执行每个预取器的 mvin 指令
        results = []
        for i, rs1 in enumerate(mvin_instructions):
            predicted_address, stride = self.prefetchers[i].execute_mvin(rs1)
            results.append((i, rs1, predicted_address, stride))
        
        return results



if __name__ == "__main__":
    # 所有的 mvin 指令，rs1 为虚拟 DRAM 地址，包含三轮数据
    mvin_instructions = [
        [1000, 1004, 1008, 1012, 1016, 1020, 1024, 1028, 1032, 1036, 1040, 1044, 1048, 1052, 1056, 1060], # 第一轮
        [1064, 1068, 1072, 1076, 1080, 1084, 1088, 1092, 1096, 1100, 1104, 1108, 1112, 1116, 1120, 1124], # 第二轮
        [1128, 1132, 1136, 1140, 1144, 1148, 1152, 1156, 1160, 1164, 1168, 1172, 1176, 1180, 1184, 1188]  # 第三轮
    ]

    # 初始化预取器组，包含16个步长预取器
    prefetcher_group = StridePrefetcherGroup(num_prefetchers=16, default_stride=4)

    # 执行每轮的 mvin 指令
    for round_num in range(3):  # 每轮16个指令
        print(f"--- 第 {round_num + 1} 轮 ---")
        
        # 从二维列表中取出对应轮次的指令列表
        round_instructions = mvin_instructions[round_num]
        
        # 执行这一轮的指令
        results = prefetcher_group.prefetch(round_instructions)

        for i, rs1, predicted_address, stride in results:
            print(f"SP[{i}] 本次访问的DRAM地址{rs1}, 设置步长{stride}, 预测的下一次访问地址{predicted_address}")

--- 第 1 轮 ---
SP[0] 本次访问的DRAM地址1000, 设置步长1000, 预测的下一次访问地址2000
SP[1] 本次访问的DRAM地址1004, 设置步长1004, 预测的下一次访问地址2008
SP[2] 本次访问的DRAM地址1008, 设置步长1008, 预测的下一次访问地址2016
SP[3] 本次访问的DRAM地址1012, 设置步长1012, 预测的下一次访问地址2024
SP[4] 本次访问的DRAM地址1016, 设置步长1016, 预测的下一次访问地址2032
SP[5] 本次访问的DRAM地址1020, 设置步长1020, 预测的下一次访问地址2040
SP[6] 本次访问的DRAM地址1024, 设置步长1024, 预测的下一次访问地址2048
SP[7] 本次访问的DRAM地址1028, 设置步长1028, 预测的下一次访问地址2056
SP[8] 本次访问的DRAM地址1032, 设置步长1032, 预测的下一次访问地址2064
SP[9] 本次访问的DRAM地址1036, 设置步长1036, 预测的下一次访问地址2072
SP[10] 本次访问的DRAM地址1040, 设置步长1040, 预测的下一次访问地址2080
SP[11] 本次访问的DRAM地址1044, 设置步长1044, 预测的下一次访问地址2088
SP[12] 本次访问的DRAM地址1048, 设置步长1048, 预测的下一次访问地址2096
SP[13] 本次访问的DRAM地址1052, 设置步长1052, 预测的下一次访问地址2104
SP[14] 本次访问的DRAM地址1056, 设置步长1056, 预测的下一次访问地址2112
SP[15] 本次访问的DRAM地址1060, 设置步长1060, 预测的下一次访问地址2120
--- 第 2 轮 ---
SP[0] 本次访问的DRAM地址1064, 设置步长64, 预测的下一次访问地址1128
SP[1] 本次访问的DRAM地址1068, 设置步长64, 预测的下一次访问地址1132
SP[2] 本次访问的DRAM地址1072, 设置步长64, 预测的下一次访问地址1136
SP[3] 本次访问的DRAM地址1076, 设置步长64, 预测的下一次访问地址1140
SP[4] 本次访问的DRA